# Предсказание стоимости жилья

В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве  — median_house_value. Для оценки качества модели буду использовать метрики RMSE, MAE и R2.

В колонках датасета содержатся следующие данные:
- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

# Подготовка данных

## Изучение данных

Выгрузим необходимые библиотеки для дальнейшей работы.

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Imputer

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder as OneHotEncoder  
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator as OneHotEncoder
    

Инициализируем локальную Spark-сессию.

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("housing - Linear Regression") \
                    .getOrCreate()

Прочитаем содержимое файла '/datasets/housing.csv'.

In [3]:
data = spark.read.load('/datasets/housing.csv', format="csv", sep=",", inferSchema=True, header="true")

Выведим типы данных колонок датасета и первые 10 строк датасета.

In [4]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
data.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

## Предобработка данных

Найдем пропущенные значения.

In [6]:
columns = data.columns

for column in columns:
    check_col = F.col(column)
    print(column, data.filter(check_col.isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Разделим выборки на обучающую и тестовую.

In [7]:
train_data, test_data = data.randomSplit([.8,.2], seed=2022)
print(train_data.count(), test_data.count())

16418 4222


Заменим пропущенные значения на медиану стоблца total_bedrooms.

In [8]:
imputer = Imputer(
    strategy="median", missingValue=float("nan"),
    inputCols=["total_bedrooms"], outputCols=["total_bedrooms"],).fit(train_data)
train_data = imputer.transform(train_data)
test_data = imputer.transform(test_data)

Трансформируем категориальный признак в числовое представление с помощью трансформера StringIndexer.

In [9]:
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx'],
                        handleInvalid='skip').fit(train_data)
train_data = indexer.transform(train_data)
test_data = indexer.transform(test_data)

Преобразуем колонку с категориальными значениями техникой One hot encoding.

In [10]:
encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe']).fit(train_data)
train_data = encoder.transform(train_data)
test_data = encoder.transform(test_data)

Соберем числовые признаки с помощью VectorAssembler. Отшкалируем числовые колонки методом StandardScaler.

In [11]:
numerical_cols = ["longitude", "latitude", "housing_median_age", "total_rooms",
                   "total_bedrooms", "population", "households", "median_income"]

In [12]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols,outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

standardScaler = StandardScaler(inputCol='numerical_features', 
                                outputCol='numerical_features_scaled').fit(train_data)
train_data = standardScaler.transform(train_data) 
test_data = standardScaler.transform(test_data) 

In [13]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
train_data.select(all_features).show(5) 
test_data = final_assembler.transform(test_data)
test_data.select(all_features).show(5) 

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (3,[2],[1.0])|     [-61.952887791441...|
|      (3,[2],[1.0])|     [-61.927977100733...|
|      (3,[2],[1.0])|     [-61.913030686308...|
|      (3,[2],[1.0])|     [-61.908048548166...|
|      (3,[2],[1.0])|     [-61.903066410024...|
+-------------------+-------------------------+
only showing top 5 rows

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (3,[2],[1.0])|     [-61.927977100733...|
|      (3,[2],[1.0])|     [-61.893102133741...|
|      (3,[2],[1.0])|     [-61.883137857458...|
|      (3,[2],[1.0])|     [-61.883137857458...|
|      (3,[2],[1.0])|     [-61.868191443033...|
+-------------------+-------------------------+
only showing top 5 rows



# Обучение моделей

Построим две модели линейной регрессии на разных наборах данных (для построения модели использю оценщик LinearRegression из библиотеки MLlib), используя все данные из файла или используя только числовые переменные, исключив категориальные.

Построим модель линейной регрессии, используя все данные из файла.

In [14]:
lr_with_cat = LinearRegression(labelCol='median_house_value', 
                      featuresCol= 'features')
model_with_cat = lr_with_cat.fit(train_data) 
predictions_with_cat = model_with_cat.transform(test_data)
predictedLabes_with_cat = predictions_with_cat.select("median_house_value", "prediction")
predictedLabes_with_cat.show(10) 

23/08/15 17:17:41 WARN Instrumentation: [2e69d92b] regParam is zero, which might cause numerical instability and overfitting.
23/08/15 17:17:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/08/15 17:17:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/08/15 17:17:42 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/08/15 17:17:42 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|152991.55876715155|
|           50800.0|214968.05218253378|
|           58100.0|142714.88166237995|
|           68400.0| 132484.3262109626|
|           72200.0| 164100.9009900242|
|           67000.0|154444.31603433192|
|           81300.0|152871.00750037655|
|           70500.0|164581.43780993763|
|           60000.0|142736.69061019877|
|          109400.0| 171314.4460876761|
+------------------+------------------+
only showing top 10 rows



Построим модель линейной регрессии, используя только числовые переменные, исключив категориальные.

In [15]:
lr_without_cat = LinearRegression(labelCol='median_house_value', 
                      featuresCol= 'numerical_features_scaled')
model_without_cat = lr_without_cat.fit(train_data) 
predictions_without_cat = model_without_cat.transform(test_data)
predictedLabes_without_cat = predictions_without_cat.select("median_house_value", "prediction")
predictedLabes_without_cat.show(10) 

23/08/15 17:17:45 WARN Instrumentation: [422bb274] regParam is zero, which might cause numerical instability and overfitting.


+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|101397.05265782354|
|           50800.0|183325.84112823103|
|           58100.0|109609.10360910231|
|           68400.0| 80433.75487934053|
|           72200.0|129999.89959631395|
|           67000.0| 120450.9910860518|
|           81300.0|118116.53813811578|
|           70500.0|130755.57313267142|
|           60000.0|110062.72904391447|
|          109400.0|118243.65155705344|
+------------------+------------------+
only showing top 10 rows



# Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. 

In [16]:
rmse_without_cat = RegressionEvaluator(labelCol='median_house_value', 
                                       metricName='rmse').evaluate(predictedLabes_without_cat)
mae_without_cat = RegressionEvaluator(labelCol='median_house_value', 
                                      metricName='mae').evaluate(predictedLabes_without_cat)
r2_without_cat = RegressionEvaluator(labelCol='median_house_value', 
                                     metricName='r2').evaluate(predictedLabes_without_cat)

print('RMSE_without_cat: ', rmse_without_cat)
print('MAE_without_cat: ', mae_without_cat) 
print('R2_without_cat: ', r2_without_cat) 

RMSE_without_cat:  69009.82619805129
MAE_without_cat:  50751.95211107678
R2_without_cat:  0.6476009347271927


In [17]:
rmse_with_cat = RegressionEvaluator(labelCol='median_house_value', 
                                    metricName='rmse').evaluate(predictedLabes_with_cat)
mae_with_cat = RegressionEvaluator(labelCol='median_house_value', 
                                   metricName='mae').evaluate(predictedLabes_with_cat)
r2_with_cat = RegressionEvaluator(labelCol='median_house_value', 
                                  metricName='r2').evaluate(predictedLabes_with_cat)

print('RMSE_with_cat: ', rmse_with_cat)
print('MAE_with_cat: ', mae_with_cat) 
print('R2_with_cat: ', r2_with_cat) 

RMSE_with_cat:  68223.20473212477
MAE_with_cat:  49726.16247158131
R2_with_cat:  0.655588921359124


In [18]:
spark.stop()

1. RMSE - метрика, которая сообщает нам квадратный корень из средней квадратичной разницы между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже RMSE, тем лучше модель соответствует набору данных. По параметру RMSE модель линейной регрессии, обученная на всех данных лучше модели, обученной только на численных данных. 
2. MAE - метрика, которая сообщает нам среднюю абсолютную разницу между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже MAE, тем лучше модель соответствует набору данных. По параметру MAE модель линейной регрессии, обученная на всех данных лучше модели, обученной только на численных данных. 
3. R2 - vетрика, которая сообщает нам долю дисперсии переменной отклика регрессионной модели, которая может быть объяснена предикторными переменными. Это значение находится в диапазоне от 0 до 1. Чем выше значение R2 , тем лучше модель соответствует набору данных. По параметру R2 модель линейной регрессии, обученная на всех данных лучше модели, обученной только на численных данных. 